## ECOPOTENTIAL-00003 Data Pipeline Coodinator - NDVI CAMARGUE

In [1]:
import owslib
from owslib.wps import monitorExecution
import uuid
from owslib.wps import WebProcessingService
import sys
import os
sys.path.append('/opt/anaconda/bin/')
import lxml.etree as etree
os.environ["TMPDIR"] = '1'
os.environ["_CIOP_APPLICATION_PATH"] = '1'
import requests
import cioppy
ciop = cioppy.Cioppy()
import shapely
import dateutil.parser
from shapely.wkt import loads 
import pandas as pd
import geopandas as gp
from datetime import datetime, timedelta
from io import BytesIO
from zipfile import ZipFile
%matplotlib inline
import matplotlib.pyplot as plt
import gdal
import numpy as np

### Common Parameters

In [2]:
process_id = 'ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9'

In [ ]:
series = 'https://catalog.terradue.com/sentinel2/description'

In [3]:
data_pipeline = 'ecopotential-00003'
wps_url_triggers = 'https://ec-ecopotential-triggers-deployer.terradue.com/zoo/'
wps_url_apps = 'https://ec-ecopotential-apps-deployer.terradue.com/zoo/'
geom = 'POLYGON%20((3.9838142131185497%2043.6616472905084194%2C%203.9868515806338043%2043.6669731264617411%2C%203.9961078879006600%2043.6851954907525126%2C%204.0073151636195155%2043.7025522352015372%2C%204.0202532489462950%2043.7186601697365518%2C%204.0347840752305819%2043.7333473986924517%2C%204.0399007377250067%2043.7375480660952363%2C%204.0411357278019624%2043.7398500717055256%2C%204.0525980996619628%2043.7590269244055250%2C%204.0635723954316463%2043.7754811066829106%2C%204.0761182065435966%2043.7907709041178421%2C%204.0901128427178062%2043.8047467919397064%2C%204.1032319206078061%2043.8166122010397103%2C%204.1192188887660368%2043.8296379791798216%2C%204.1364621570852016%2043.8409483171376806%2C%204.1547784063243762%2043.8504229707385278%2C%204.1739729100056673%2043.8579612116126683%2C%204.1938416046197862%2043.8634828980741176%2C%204.2141732590968788%2043.8669293271350114%2C%204.2347517204782044%2043.8682638585977145%2C%204.2511398708882044%2043.8684799179977034%2C%204.2705615707154685%2043.8677916991206374%2C%204.2898247867769737%2043.8652217620842819%2C%204.3087476376795673%2043.8607943719815978%2C%204.3271514557244792%2043.8545513317948874%2C%204.3448624738701778%2043.8465515876961192%2C%204.3467416693653842%2043.8454719876547401%2C%204.3508464740689146%2043.8458866651010126%2C%204.3556227975340667%2043.8503142000825079%2C%204.3708019362184904%2043.8619014776782024%2C%204.3870169954765821%2043.8719882564770671%2C%204.4041201461587995%2043.8804825775224430%2C%204.4219554625802830%2043.8873069999468086%2C%204.4336323148702830%2043.8911466558468035%2C%204.4501257352262336%2043.8957973001305319%2C%204.4669567262525822%2043.8990189686986483%2C%204.4840017226365569%2043.9007880095580205%2C%204.4863318889065837%2043.9009293013207014%2C%204.5030267648200306%2043.9044608058976564%2C%204.5213564028514535%2043.9065995408354084%2C%204.5398051448072811%2043.9070396973351436%2C%204.5582159227948598%2043.9057775280164577%2C%204.5764319921370991%2043.9028237786676314%2C%204.5872599742370985%2043.9005509109676382%2C%204.6071875776229616%2043.8952826632088247%2C%204.6264671193599272%2043.8879913614344446%2C%204.6448938199585035%2043.8787544509009990%2C%204.6622719584626111%2043.8676700423337991%2C%204.6724964366226116%2043.8603810299337979%2C%204.6881445946551024%2043.8480033046381905%2C%204.6991947750231997%2043.8373090614257563%2C%204.7257918332406517%2043.8352885238605410%2C%204.7440859043423869%2043.8330470614121950%2C%204.7620959536960319%2043.8291310285137214%2C%204.7796690322220350%2043.8235736817995374%2C%204.7966559017849155%2043.8164222166464157%2C%204.8129123025876517%2043.8077373663700129%2C%204.8283001782878747%2043.7975928864514970%2C%204.8426888484316679%2043.7860749281742798%2C%204.8617897215716681%2043.7692838736742758%2C%204.8761382053899966%2043.7553378251092013%2C%204.8890213750738694%2043.7400278545879715%2C%204.9003102793518014%2043.7235072039147070%2C%204.9098919244034835%2043.7059412329319201%2C%204.9101276959190221%2043.7053824401972548%2C%204.9213855823003305%2043.6968562434258416%2C%204.9350338141830159%2043.6843768820800520%2C%204.9474709984365450%2043.6706902045808931%2C%204.9585907944326024%2043.6559132349954737%2C%204.9589933399337669%2043.6553233870890764%2C%204.9813961228206587%2043.6345174182812059%2C%204.9950736102332334%2043.6205068216403546%2C%205.0073155707525832%2043.6052259550812735%2C%205.0180046742216486%2043.5888212744327035%2C%205.0270384734721976%2043.5714500064520820%2C%205.0343303862053217%2043.5532786419230717%2C%205.0378502557043552%2043.5412052155930880%2C%205.0426526009296140%2043.5376619681182717%2C%205.0575328001079338%2043.5241812460943507%2C%205.0709863483047455%2043.5092764738790905%2C%205.0828776505059530%2043.4930978730176960%2C%205.0930868571918619%2043.4758085036620159%2C%205.1015110722711023%2043.4575826211272442%2C%205.1080653901451161%2043.4386039196153106%2C%205.1126837514508621%2043.4190636808060617%2C%205.1153196088569448%2043.3991588459761175%2C%205.1159463962027587%2043.3790900310759824%2C%205.1158303422127585%2043.3729813259759851%2C%205.1146191253387263%2043.3544786023268998%2C%205.1116995404204859%2043.3361675714259533%2C%205.1070966825917212%2043.3182056247453104%2C%205.1008501154670522%2043.3007471532226589%2C%205.0930135310751270%2043.2839422202023059%2C%205.0836542883517062%2043.2679352715738403%2C%205.0728528341595887%2043.2528638941953218%2C%205.0607020118119657%2043.2388576332729073%2C%205.0473062630427643%2043.2260368788620823%2C%205.0418223352127640%2043.2212551992620817%2C%205.0244546835631887%2043.2077197501405195%2C%205.0163569347703652%2043.2020968231571345%2C%205.0104011336464680%2043.1978366025550216%2C%205.0095275470269280%2043.1971890343373914%2C%204.9949018723933483%2043.1749910078431327%2C%204.9829625163884712%2043.1587207262497614%2C%204.9694442852375573%2043.1437364774701564%2C%204.9544848182869217%2043.1301908274860679%2C%204.9382364291957543%2043.1182216948157091%2C%204.9208645551154273%2043.1079509462609067%2C%204.9025460722486072%2043.0994831560889224%2C%204.8834674949387127%2043.0929045412826710%2C%204.8638230766261765%2043.0882820837003067%2C%204.8438128320070879%2043.0856628480822508%2C%204.8236405005321394%2043.0850735028494114%2C%204.8035114719810226%2043.0865200485718134%2C%204.7671026156010221%2043.0909920115718137%2C%204.7436655677188435%2043.0953010216248913%2C%204.7291590605503107%2043.0988730630087531%2C%204.7084434012456020%2043.0982971082500654%2C%204.6627011147870983%2043.0967755224220852%2C%204.6435604568008850%2043.0970554360972926%2C%204.6245342341763145%2043.0991639883141104%2C%204.5738916673963139%2043.1072456547141130%2C%204.5594793824222251%2043.1100919684504262%2C%204.5042298003722250%2043.1231284516504232%2C%204.4883210128393864%2043.1275833047372643%2C%204.4728341747748601%2043.1333359635619686%2C%204.4578749587740019%2043.1403471755420753%2C%204.4222207843840025%2043.1588909902420781%2C%204.4030475886136369%2043.1702633539185712%2C%204.3852590973536243%2043.1836988170788629%2C%204.3690763068553125%2043.1990304635438349%2C%204.3664107596743698%2043.2018539059637590%2C%204.3567228951661985%2043.2026961461102275%2C%204.2962590875004576%2043.2070287875862107%2C%204.2778393170932034%2043.2092110195418400%2C%204.2597009321106549%2043.2130903324177140%2C%204.2419999453720871%2043.2186333592663914%2C%204.2248886075242744%2043.2257924231173476%2C%204.2085140974953159%2043.2345059470586506%2C%204.1456271867038899%2043.2718114005216847%2C%204.0933119560514006%2043.2903066045541678%2C%204.0755069440457428%2043.2975822603457985%2C%204.0584831668042503%2043.3065346135644802%2C%204.0423980745272177%2043.3170808653232484%2C%204.0274004356674302%2043.3291234750217882%2C%204.0136289609944740%2043.3425510624838779%2C%204.0012110206809526%2043.3572394380948793%2C%203.9902614662760771%2043.3730527514116559%2C%203.9808815684620078%2043.3898447476216802%2C%203.9731580804174671%2043.4074601202304535%2C%203.9604070798174664%2043.4408185482304532%2C%203.9533703182447022%2043.4630282369200032%2C%203.9489640893353979%2043.4859055495226059%2C%203.9451020825953984%2043.5149941196226067%2C%203.9435032994941150%2043.5338089077261188%2C%203.9436860216770429%2043.5526906176960580%2C%203.9456486203935492%2043.5714709415767345%2C%203.9493736014135905%2043.5899824751486804%2C%203.9548277609679392%2043.6080602101379071%2C%203.9619624817202377%2043.6255430050681170%2C%203.9707141661326482%2043.6422750216450410%2C%203.9810048033621563%2043.6581071138692494%2C%203.9838142131185497%2043.6616472905084194))'
product_type = 'S2MSI2Ap'
cloud_coverage = '100['
tg_quotation = 'No'
api_key = 'AKCp5aTSAokLQTzhVH52Q8tcfobtaVQtJ3xPiGgB8AdxSPvDpUXU1DW2uYby4EjJcEiJtaoPu'
recovery = 'No'
pa_code = 'CA'
pa_name = 'Camargue'
resolution = '10'
plot = 'False'
flag_expr = 'saturated_l1a_B4'
_T2Username = 'ecopotential-00003'

### WPS

In [4]:
wps = WebProcessingService(wps_url_apps, verbose=False, skip_caps=False)

In [5]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'ec_ecopotential_dcs_s2_ndvi_boa_ndvi_boa_1_9')
(1, 'ec_ecopotential_dcs_snap_hydroperiod_preproc_dcs_snap_hydroperiod_preproc_0_2')
(2, 'GetStatus')


In [6]:
process = wps.describeprocess(process_id)

In [7]:
for input in process.dataInputs:
    print(input.identifier)

source
pa_code
pa_name
resolution
plot
flag_expr
_T2Username


### Check trigger coordinator

In [200]:
wps = WebProcessingService(wps_url_triggers, verbose=False, skip_caps=False)

In [201]:
for index, elem in enumerate(wps.processes):
    print(index, elem.identifier)

(0, 'coordinator_ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12')
(1, 'ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12')
(2, 'coordinator_ec_ecopotential_tg_ecopotential_00003_via_tg_ecopotential_00003_1_12')
(3, 'GetStatus')
(4, 'ec_ecopotential_tg_ecopotential_00003_via_tg_ecopotential_00003_1_12')


### Feed the queue

In [10]:
co_trigger_queue_process_id = 'coordinator_ec_ecopotential_tg_ecopotential_00003_queue_tg_ecopotential_00003_1_12'

In [11]:
process = wps.describeprocess(co_trigger_queue_process_id)

In [12]:
process.title

'Trigger for ecopotential-00003 Coordinator'

In [13]:
process.abstract

'Coordinator: Trigger for ecopotential-00003'

In [14]:
for input in process.dataInputs:
    print(input.identifier)

series
data_pipeline
wps_url
process_id
update
geom
product_type
cloud_coverage
tg_quotation
api_key
recovery
mode
pa_code
pa_name
resolution
plot
flag_expr
t2_coordinator_date_start
t2_coordinator_date_stop
t2_coordinator_period
_T2Username


##### Parameters


In [172]:
start_queue = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -10, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'
end_queue = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -9, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'

In [173]:
start_queue


'${coord:formatTime(coord:dateOffset(coord:nominalTime(), -10, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'

In [174]:
mode = 'Queue'

In [175]:
coordinator_date_start = '2018-01-01T00:00Z'
coordinator_date_stop = '2018-12-31T00:00Z'
coordinator_period = '0 0 * * *'

In [176]:
inputs = [('series', series),
        ('data_pipeline', data_pipeline),
        ('wps_url', wps_url_apps),
        ('process_id', process_id),
        ('update', '%s/%s' % (start_queue, end_queue)),
        ('geom', geom),
        ('product_type', product_type),
        ('cloud_coverage', cloud_coverage),
        ('tg_quotation', tg_quotation),
        ('api_key', api_key),
        ('recovery', recovery),
        ('mode', mode),
        ('pa_code', pa_code),
        ('pa_name', pa_name),
        ('resolution', resolution),
        ('plot', plot),
        ('flag_expr', flag_expr),
        ('t2_coordinator_date_start', coordinator_date_start), 
        ('t2_coordinator_date_stop', coordinator_date_stop),
        ('t2_coordinator_period', coordinator_period),
        ('_T2Username', data_pipeline)]

In [177]:
for output in process.processOutputs:
    print(output.identifier)

coordinatorIds


In [178]:
execution = owslib.wps.WPSExecution(url=wps.url)

In [179]:
execution_request = execution.buildRequest(co_trigger_queue_process_id, 
                                           inputs, 
                                           output = [('coordinatorIds', False)])

In [180]:
execution_response = execution.submitRequest(etree.tostring(execution_request, pretty_print=True))

In [ ]:
if False:
    print(etree.tostring(execution_request, pretty_print=True))

In [182]:
execution.parseResponse(execution_response)

In [193]:
if False:
    print(etree.tostring(execution_response, pretty_print=True))

In [184]:
execution.statusLocation

'http://ec-ecopotential-triggers-deployer.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=4b714248-6dbb-11e8-a2c2-0242ac110005&RawDataOutput=Result'

In [185]:
monitorExecution(execution)

In [186]:
execution.isSucceded()

True

In [187]:
for output in execution.processOutputs:
    print(output.identifier)

coordinatorIds
result_osd
QUOTATION


In [192]:
execution.processOutputs[0].value()

AttributeError: 'Output' object has no attribute 'value'

### Process Queue (mode Via)

In [218]:
co_trigger_via_process_id = 'coordinator_ec_ecopotential_tg_ecopotential_00003_via_tg_ecopotential_00003_1_12'

In [219]:
process = wps.describeprocess(co_trigger_via_process_id)

In [220]:
process.title

'Trigger for ecopotential-00003 Coordinator'

In [221]:
process.abstract

'Coordinator: Trigger for ecopotential-00003'

In [222]:
for input in process.dataInputs:
    print(input.identifier)

series
data_pipeline
wps_url
process_id
update
api_key
recovery
t2_coordinator_date_start
t2_coordinator_date_stop
t2_coordinator_period
_T2Username


#### Parameters

In [223]:
start_via = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -1, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'
end_via = '${coord:formatTime(coord:dateOffset(coord:nominalTime(), -0, \'DAY\'), "yyyy-MM-dd\'T\'HH:mm:ss\'Z\'")}'

In [224]:
series_via = 'https://catalog.terradue.com//ecopotential-00003/series/source-queue/description'
mode = 'Via'

In [225]:
coordinator_date_start = '2018-06-11T00:00Z'
coordinator_date_stop = '2018-12-31T00:00Z'
coordinator_period = '0 0 * * *'

In [226]:
inputs = [('series', series_via),
        ('data_pipeline', data_pipeline),
        ('wps_url', wps_url_apps),
        ('process_id', process_id),
        ('update', '%s/%s' % (start_via, end_via)),
        ('api_key', api_key),
        ('recovery', recovery),
        ('t2_coordinator_date_start', coordinator_date_start), 
        ('t2_coordinator_date_stop', coordinator_date_stop),
        ('t2_coordinator_period', coordinator_period),
        ('_T2Username', data_pipeline)]

In [227]:
execution = owslib.wps.WPSExecution(url=wps.url)

execution_request = execution.buildRequest(co_trigger_via_process_id, 
                                           inputs, 
                                           output = [('coordinatorIds', False)])

execution_response = execution.submitRequest(etree.tostring(execution_request))

execution.parseResponse(execution_response)

In [228]:
execution.statusLocation

'http://ec-ecopotential-triggers-deployer.terradue.com/zoo-bin/zoo_loader.cgi?request=Execute&service=WPS&version=1.0.0&Identifier=GetStatus&DataInputs=sid=7c6e63d4-6e11-11e8-b46d-0242ac110005&RawDataOutput=Result'

In [229]:
monitorExecution(execution)
execution.isSucceded()

True

### 